# Maestría en Ciencia de Datos, ITAM

**Curso de Optimización 2 2021-1,"Optimización Avanzada"
Prof. Erick Palacios Moreno**

* Equipo 5:  
>MIGUEL LOPEZ  
CARLOS LOPEZ  
JOSÉ ZARATE 


## RESOLVER EL PROBLEMA DE TRAVELING SALESMAN PROBLEM (A.K.A. TSP) PARA DISTINTAS CIUDADES UTILIZANDO EL MÉTODO DE HILL CLIMBING (RANDOM-RESTART)

* Distancia Euclideana.
> Dataset: National Traveling Salesman Problems, CANADA

Para ello, realizaremos una ejecución para un subgrupo del dataset con las primeras 17 ciudades con el objetivo de realizar una _test_ estandar en tiempo razonable.

-------

### Hill Climbing 

El algoritmo de hill-climbing realiza seguimiento de un estado actual y en cada iteración se mueve al estado vecino con el valor más alto o menor, según sea el caso, es decir, se dirige en la dirección que proporciona el ascenso o descenso de mayor inclinación. Termina cuando alcanza un máximo o mínimo local. Hill climbing no mira hacia el futuro más allá de los vecinos inmediatos del estado actual. Esto se asemeja a tratar de encontrar la cima del Monte Everest en una espesa niebla mientras sufre de amnesia. Toma en cuenta que una forma de utilizar hill-climbing  consiste en utilizar el negativo de una función de coste heurística como función objetivo; que ascenderá localmente al estado con menor distancia heurística a la meta.

Los problemas que tiene Hill Climbing son:

MÁXIMA LOCAL: Un máximo local es un pico que es más alto que los estados vecinos pero menor que el máximo global. Cuando este algoritmo alcanzan las proximidades de un máximo local no podrá escapar de este, amenos que estemos tratando con un problema convexo.

CORDILLERAS: Las cordilleras son un reto para los algoritmos de hill climbing que optimizan en espacios continuos. Debido a que los algoritmos de hill climbing ajustan solo un elemento en el vector a la vez, a cada paso se va a mover en una dirección alineada con el eje. Si la función objetivo determina una cordillera estrecha que asciende en una dirección no alineada con el eje, entonces el algoritmo de hill climbing solo puede ascender la cordillera en zigzag. Si los lados de la cordillera son muy pronunciados, entonces el algoritmo puede verse forzado a realizar pasos muy pequeños mientras zigzaguea hacia una mejor posición.

MESETA: Se encuentra una meseta cuando el espacio de búsqueda es plano o lo suficientemente plano como para que el valor devuelto por la función objetivo sea indistinguible del valor devuelto para las regiones cercanas debido a la precisión utilizada por la máquina para representar la misma.

-------
**MODIFICACIONES DERIVADAS DE LA ETAPA DE EXPERIMENTACIÓN:**

Se desarrollaron diversos experimentos para analizar la eficiencia del algoritmo, la tolerancia y el punto inicial. Observamos que una de las mayores dificultades a resolver era que nuestro algoritmo era ineficiente para un númer mayor a 9 ciudades para cualquier punto inicial y tolerancia asignada (norma-distncia entre dos rutas para elegir la de mayor eficiencia). Se realizó una recodificación del código a partir de consultar otros algoritmos desarrollados por la comunidad (ver README de nuestro repo) bajo los cuales logramos tiempos sumamente eficientes incluso con más de 100 ciudades.

En una segunda etapa buscaremos optimizar el código en la siguiente práctica por medio de un análisis de perfilamiento y la consecuente reprogramación; finalmente se implementará el cómputo en paralelo. 

______________

A continuación se muestra la descripción de la instancia utilizada:

In [4]:
%%bash
lscpu

Arquitectura:                        x86_64
modo(s) de operación de las CPUs:    32-bit, 64-bit
Orden de los bytes:                  Little Endian
Address sizes:                       39 bits physical, 48 bits virtual
CPU(s):                              12
Lista de la(s) CPU(s) en línea:      0-11
Hilo(s) de procesamiento por núcleo: 2
Núcleo(s) por «socket»:              6
«Socket(s)»                          1
Modo(s) NUMA:                        1
ID de fabricante:                    GenuineIntel
Familia de CPU:                      6
Modelo:                              158
Nombre del modelo:                   Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz
Revisión:                            13
CPU MHz:                             2926.443
CPU MHz máx.:                        4500.0000
CPU MHz mín.:                        800.0000
BogoMIPS:                            5199.98
Virtualización:                      VT-x
Caché L1d:                           192 KiB
Caché L1i:               

In [3]:
%%bash
lshw -C memory

  *-memory
       descripción: Memoria de sistema
       id físico: 0
       tamaño: 8GiB
  *-memory NO RECLAMADO
       descripción: RAM memory
       producto: Cannon Lake PCH Shared SRAM
       fabricante: Intel Corporation
       id físico: 14.2
       información del bus: pci@0000:00:14.2
       versión: 10
       anchura: 64 bits
       reloj: 33MHz (30.3ns)
       capacidades: cap_list
       configuración: latency=0
       recursos: memoria:a441a000-a441bfff memoria:a4422000-a4422fff


AVISO: debería ejecutar este programa como superusuario.
AVISO: la salida puede ser incompleta o imprecisa, debería ejecutar este programa como superusuario.


In [1]:
import os
import time
import numpy as np
import pandas as pd
os.chdir("../")
from src.hill_cg.hill import best_solution
from src.hill_cg.hill_mul import parallel_hc

Enseguida ejecutamos un ejemplo donde se meustra la comparación entre nuestro código sin paralelizar contra nuestro código paralelizando.

Nuestro primer ejemplo lo corremos con 30 ciudades extraidas de nuestro dataset ejemplo, el cual contiene 4,663 ciudades canadienses.En primer lugar correremos la función `best_solution` el cual no utiliza `Pool` para déspues correr nuestro código con computo en paralelo. 

Ambas funciones regresan los mismos tres resultados: distancia más corta encontrada, la permutación correspondiente a dicha distancia y el tiempo de ejecución.

In [3]:
# Cargamos los datos 

dat = pd.read_csv("datasets/ca4663.tsp", names = ["index","uno","dos"], sep = " ")
dat.drop(['index'],axis = 1, inplace = True)
dat.dropna(inplace = True)
dat1 = dat.to_numpy()
tsp_cities = dat1[0:30,:]

## Ejemplo 12 ciudades 

Los primeros tres parámetros que se ingresan en cada función son idénticos equivalentes, mientras que el cuarto parámetro, que para `parallel_hc` es 100 y para `best_solution` es 800,  estas son  equivalentes. Ya que está instancia tiene, como se muestra en la descripción, ocho workers posibles, en cambio `best_solution` solo corre un worker a la vez.  

In [6]:
%%time
parallel_hc(tsp_cities, 0, 1e-1, 20)

Process ForkPoolWorker-12:
Process ForkPoolWorker-11:
Process ForkPoolWorker-10:
Process ForkPoolWorker-9:
Process ForkPoolWorker-7:
Process ForkPoolWorker-4:
Process ForkPoolWorker-2:
Process ForkPoolWorker-1:
Process ForkPoolWorker-3:
Process ForkPoolWorker-8:
Traceback (most recent call last):
Traceback (most recent call last):


In [4]:
%%time
best_solution(tsp_cities, 0, 1e-1, 160)

KeyboardInterrupt: 

## Ejemplo 20 ciudades

In [28]:
tsp_cities_dos = dat1[0:20,:]

In [30]:
%%time
parallel_hc(tsp_cities_dos, 0, 1e-1, 100)

CPU times: user 4.67 ms, sys: 36 ms, total: 40.7 ms
Wall time: 9.82 s


(3329.1623916962017,
 [0, 1, 5, 6, 12, 13, 11, 15, 17, 18, 19, 16, 14, 7, 2, 4, 9, 10, 8, 3, 0],
 8.481815576553345)

In [32]:
%%time
best_solution(tsp_cities_dos, 0, 1e-1, 800)

CPU times: user 20.8 s, sys: 3.99 ms, total: 20.8 s
Wall time: 20.8 s


(3329.162391696201,
 [0, 3, 8, 10, 9, 4, 2, 7, 14, 16, 19, 18, 17, 15, 11, 13, 12, 6, 5, 1, 0],
 20.783585786819458)

In [4]:
a = os.getcwd()
a

'/shared_volume/OptAvzda-2021-hill-eureka'

In [6]:
from hill_cg.hill import best_solution

In [5]:
import os 
print(os.getcwd())

/shared_volume/OptAvzda-2021-hill-eureka
